# Face detection using Haarcascade


In [20]:
import cv2
import numpy as np

# Reading

In [9]:
cap=cv2.VideoCapture(0)
face_cascade=cv2.CascadeClassifier(r'C:\Users\anshu\OneDrive\Desktop\haarcascade_frontalface_alt.xml')
while True:
    ret,frame=cap.read()
    gray_image=cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)
    if ret== False:
        continue
    
    faces=face_cascade.detectMultiScale(gray_image,1.3,2)
     
    
    for (x,y,w,h) in faces:
        cv2.rectangle(frame,(x,y),(x+w,y+h),(255,255,0),)
    cv2.imshow('frame',frame)
    key_pressed=cv2.waitKey(1) & 0xFF
    if key_pressed == ord('q'):
        break
cap.release()
cv2.destroyAllWindows()
    

# Generating Face Data


In [31]:
import cv2
import numpy as np
cap=cv2.VideoCapture(0)
face_cascade=cv2.CascadeClassifier(r'C:\Users\anshu\OneDrive\Desktop\haarcascade_frontalface_alt.xml')

skip=0
face_data=[]
dataset_path=r'C:\Users\anshu\OneDrive\Desktop\ '


face_section = np.zeros((100,100,3))

file_name=input("Enter person's name")

while True:
    ret,frame=cap.read() 
    
    if ret == False:
        continue
        
    cv2.imshow('frame', frame)
        
    
    faces = face_cascade.detectMultiScale(frame,1.2,3)
     
    faces = sorted(faces,key=lambda f:f[2]*f[3])
    
    if(len(faces)!=0):
        for face in faces[-1:]:
            x,y,w,h=face
            cv2.rectangle(frame,(x,y),(x+w,y+h),(255,255,0),2)
        
        #Crop out require face: Region of Interest
            offset=10
            face_section=frame[y-offset:y+h+offset,x-offset:x+w+offset]
            face_section=cv2.resize(face_section,(100,100))
        
            skip+=1
            if skip%10==0:
                face_data.append(face_section)
                print(len(face_data))
            
    cv2.imshow("cropped face", face_section)
    cv2.imshow('frame', frame)
        
    key_pressed=cv2.waitKey(1) & 0xFF
    if key_pressed == ord('q'):
        break
    


face_data=np.asarray(face_data)
face_data=face_data.reshape((face_data.shape[0],-1))

np.save(dataset_path+file_name+".npy",face_data)

cap.release()
cv2.destroyAllWindows()



Enter person's namesober
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36


# Testing on the new data

In [33]:
import cv2
import numpy as np 
import os 


#KNN ALGORITHM
def dist(x1,x2):
    return np.sqrt(sum((x1-x2)**2))

def knn(X,Y,query,k):
    val=[]
    for i in range(X.shape[0]):
        distance=dist(query,X[i])
        val.append((distance,Y[i]))
        
    val=sorted(val)
    val=val[:k]
    val=np.array(val)
    new_val= np.unique(val[:,1],return_counts=True)
    #print(val[0]) 
    idx=new_val[1].argmax()
    val=new_val[0][idx]
    return int(val)


#Init Camera
cap = cv2.VideoCapture(0)

# Face Detection
face_cascade = cv2.CascadeClassifier(r'C:\Users\anshu\OneDrive\Desktop\haarcascade_frontalface_alt.xml')

skip = 0
dataset_path = r'C:\Users\anshu\OneDrive\Desktop\\'

face_data = [] 
labels = []

class_id = 0 # Labels for the given file
names = {} #Mapping btw id - name


# Data Preparation
for fx in os.listdir(dataset_path):
    if fx.endswith('.npy'):
        #Create a mapping btw class_id and name
        names[class_id] = fx[:-4]
        print("Loaded "+fx)
        data_item = np.load(dataset_path+fx)
        face_data.append(data_item)
        
        #Create Labels for the class
        target = class_id*np.ones((data_item.shape[0],))
        class_id += 1
        labels.append(target)

face_dataset = np.concatenate(face_data,axis=0)
face_labels = np.concatenate(labels,axis=0).reshape((-1,1))

print(face_dataset.shape)
print(face_labels.shape)


# Testing 

while True:
    ret,frame = cap.read()
    if ret == False:
        continue

    faces = face_cascade.detectMultiScale(frame,1.3,5)
    if(len(faces)==0):
        continue

    for face in faces:
        x,y,w,h = face
        #Get the face ROI
        offset = 10
        face_section = frame[y-offset:y+h+offset,x-offset:x+w+offset]
        face_section = cv2.resize(face_section,(100,100))

        #Predicted Label (out)
        out = knn(face_dataset,face_labels,face_section.flatten(),5)

        #Display on the screen the name and rectangle around it
        pred_name = names[int(out)]
        cv2.putText(frame,pred_name,(x,y-10),cv2.FONT_HERSHEY_SIMPLEX,1,(255,0,0),2,cv2.LINE_AA)
        cv2.rectangle(frame,(x,y),(x+w,y+h),(0,255,255),2)

    cv2.imshow("Faces",frame)

    key = cv2.waitKey(1) & 0xFF
    if key==ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


Loaded  sober.npy
Loaded image_datasetansh.npy
Loaded image_datasetanshuljain.npy
Loaded image_datasetyog.npy
Loaded image_datasetyogyata .npy
(179, 30000)
(179, 1)
